In [33]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC, RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, auc, roc_curve
import category_encoders as ce
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/classification/hotel_cancellation/cleaned_hotel_bookings.csv')
df

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,...,required_car_parking_spaces,total_of_special_requests,Date_sin_dayofweek,Date_cos_dayofweek,Date_sin_dayofyear,Date_cos_dayofyear,reservation_status_date_sin_dayofweek,reservation_status_date_cos_dayofweek,reservation_status_date_sin_dayofyear,reservation_status_date_cos_dayofyear
0,Resort Hotel,0,342,27,0,0,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,8.660254e-01,-0.5,1.716633e-02,-0.999853
1,Resort Hotel,0,737,27,0,0,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,8.660254e-01,-0.5,1.716633e-02,-0.999853
2,Resort Hotel,0,7,27,0,1,1,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,1.224647e-16,-1.0,1.224647e-16,-1.000000
3,Resort Hotel,0,13,27,0,1,1,0.0,0,BB,...,0,0,8.660254e-01,-0.5,0.017166,-0.999853,1.224647e-16,-1.0,1.224647e-16,-1.000000
4,Resort Hotel,0,14,27,0,2,2,0.0,0,BB,...,0,1,8.660254e-01,-0.5,0.017166,-0.999853,-8.660254e-01,-0.5,-1.716633e-02,-0.999853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119201,City Hotel,0,23,35,2,5,2,0.0,0,BB,...,0,0,8.660254e-01,-0.5,-0.848351,-0.529434,8.660254e-01,-0.5,-9.057023e-01,-0.423914
119202,City Hotel,0,102,35,2,5,3,0.0,0,BB,...,0,2,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304
119203,City Hotel,0,34,35,2,5,2,0.0,0,BB,...,0,4,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304
119204,City Hotel,0,109,35,2,5,2,0.0,0,BB,...,0,0,1.224647e-16,-1.0,-0.857315,-0.514793,1.224647e-16,-1.0,-9.128459e-01,-0.408304


In [ ]:
out_column = 'is_canceled'
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
fig = go.Figure(data=go.Heatmap(z=df[numerical].corr(),x=numerical,y=numerical))
fig.show()

In [ ]:
df[out_column].value_counts()

0    75011
1    44195
Name: is_canceled, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42, stratify=df[out_column])
encoder = ce.CatBoostEncoder(verbose=1, cols=categorical+binary)
encoder.fit(X_train[categorical+binary], Y_train)
X_train[categorical+binary] = encoder.transform(X_train[categorical+binary])
X_test[categorical+binary] = encoder.transform(X_test[categorical+binary])

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((95364, 28), (23842, 28), (95364,), (23842,))

In [ ]:
X_train

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,...,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,deposit_type,customer_type,adr,required_car_parking_spaces,total_of_special_requests,sin_dayofyear,cos_dayofyear
8276,0.279638,343.0,0.0,4.0,1.0,0.0,0.0,0.343139,0.187751,0.221574,...,0.0,0.0,0.391398,0.284361,0.253581,56.5,0.0,0.0,-0.986731,-0.162366
105716,0.416719,147.0,1.0,3.0,2.0,0.0,0.0,0.374189,0.342545,0.410175,...,0.0,0.0,0.391398,0.284361,0.407911,75.0,0.0,0.0,0.685548,0.728028
99328,0.416719,181.0,0.0,2.0,1.0,0.0,0.0,0.374189,0.342545,0.410175,...,0.0,0.0,0.391398,0.284361,0.253581,120.0,0.0,0.0,-0.983798,0.179281
27590,0.279638,11.0,1.0,4.0,2.0,1.0,0.0,0.343139,0.154148,0.176183,...,0.0,0.0,0.294698,0.284361,0.407911,229.2,1.0,0.0,-0.848351,-0.529434
9038,0.279638,65.0,2.0,5.0,2.0,1.0,0.0,0.374189,0.154148,0.176183,...,0.0,0.0,0.315731,0.284361,0.407911,76.2,0.0,0.0,-0.919720,0.392574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28916,0.279638,214.0,0.0,4.0,2.0,0.0,0.0,0.343139,0.610309,0.221574,...,0.0,0.0,0.294698,0.284361,0.253581,81.0,0.0,0.0,-0.983798,0.179281
74670,0.416719,1.0,0.0,1.0,1.0,0.0,0.0,0.374189,0.129630,0.221574,...,0.0,24.0,0.315731,0.284361,0.407911,0.0,0.0,2.0,0.697944,-0.716152
45898,0.416719,84.0,1.0,2.0,1.0,0.0,0.0,0.374189,0.610309,0.410175,...,0.0,0.0,0.391398,0.284361,0.253581,56.0,0.0,0.0,-0.565345,0.824855
5786,0.279638,115.0,2.0,6.0,2.0,0.0,0.0,0.374189,0.367978,0.410175,...,0.0,0.0,0.315731,0.284361,0.407911,73.0,0.0,1.0,0.733885,-0.679273


In [ ]:
hp = [4.8e+01, 1.0e-05, 1.0e-05, 2.3e+01]
hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
clf = RFC(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3], bootstrap=True, random_state=1)
clf.fit(X_train, Y_train)

RandomForestClassifier(max_features=23, min_samples_leaf=1e-05,
                       min_samples_split=1e-05, n_estimators=48, n_jobs=-1,
                       random_state=1)

In [34]:
hp = [5.10000000e+01, 3.45813505e-02, 2.00000000e+01, 1.00000000e-05,
       1.16451671e+00]
hp = [np.round(hp[0]).astype(int), hp[1], np.round(hp[2]).astype(int), hp[3], hp[4]]
clf = GBC(n_estimators=hp[0], min_samples_split=hp[1], max_features=hp[2], min_samples_leaf=hp[3], learning_rate=hp[4], random_state=1)
clf.fit(X_train, Y_train)

GradientBoostingClassifier(learning_rate=1.16451671, max_features=20,
                           min_samples_leaf=1e-05,
                           min_samples_split=0.0345813505, n_estimators=51,
                           random_state=1)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
score = cross_val_score(clf, X_train, Y_train, cv=kf, scoring='accuracy').mean()
score

0.975923812303453

In [35]:
pred = clf.predict_proba(X_train)[:,1]
auc = roc_auc_score(Y_train, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_train, np.round(pred)), 'PRE': precision_score(Y_train, np.round(pred)), 'REC': recall_score(Y_train, np.round(pred)), 'F1':f1_score(Y_train, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.995332,0.977665,0.9937,0.945752,0.969133


In [36]:
pred = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(Y_test, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_test, np.round(pred)), 'PRE': precision_score(Y_test, np.round(pred)), 'REC': recall_score(Y_test, np.round(pred)), 'F1':f1_score(Y_test, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.993957,0.974541,0.989301,0.941509,0.964814


In [ ]:
confusion_matrix(Y_test, np.round(pred))

array([[14965,    38],
       [  471,  8368]])

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import fbeta_score, make_scorer


result = permutation_importance(clf, X_test, Y_test, n_repeats=10, random_state=42, n_jobs=2, scoring='f1')
fig = go.Figure()
# Use x instead of y argument for horizontal plot
for i in range(X_train.shape[1]):
    fig.add_trace(go.Box(x=result['importances'][i], name=X_test.columns[i]))
fig.show()

In [ ]:
import warnings
import traceback
import dask
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier as GBC, RandomForestClassifier as RFC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import category_encoders as ce
import optimization.numpy_version.single_objective.continuous as co

def _cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
    clf = RFC(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3],
               bootstrap=True, random_state=1)
    score = cross_val_score(clf, X_train, Y_train, cv=kf, scoring='accuracy').mean()
    return hp, -score

def cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], np.round(hp[2]).astype(int), hp[3], hp[4]]
    clf = GBC(n_estimators=hp[0], min_samples_split=hp[1], max_features=hp[2], min_samples_leaf=hp[3],
               learning_rate=hp[4], random_state=1)
    score = cross_val_score(clf, X_train, Y_train, cv=kf, scoring='accuracy').mean()
    return hp, -score

def objf(pop, dim=None):
    pop = np.clip(pop, lb, ub)
    population, fitness = [], []
    for hp in pop:
        _hp, mse = dask.delayed(cv_score, nout=2)(hp)
        population.append(_hp), fitness.append(mse)
    population, fitness = dask.compute(population, fitness)
    population, fitness = np.asarray(population), np.asarray(fitness)
    return population, fitness  # duration =  27.99079418182373

df = pd.read_csv('cleaned_hotel_bookings.csv')
########################################################################################################################
out_column = 'is_canceled'
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
########################################################################################################################
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42, stratify=df[out_column])
encoder = ce.CatBoostEncoder(verbose=1, cols=categorical+binary)
encoder.fit(X_train[categorical+binary], Y_train)
X_train[categorical+binary] = encoder.transform(X_train[categorical+binary])
X_test[categorical+binary] = encoder.transform(X_test[categorical+binary])
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
print(X_train.shape, X_test.shape)
########################################################################################################################
def ccf(population):
    return population

#lb, ub = [2, 1e-5, 1e-5, 1], [51, 1, 0.5, X_train.shape[1]]
lb, ub = [2, 1e-5, 1, 1e-5, 1e-5], [51, 1, X_train.shape[1], 0.5, 2]
dim = len(lb)
optimizer = co.CPSO(objf, ccf, dim, lb, ub, 40, 5000)